In [12]:
require(tidyverse, warn.conflicts = FALSE);
require(cowplot, warn.conflicts = FALSE); # for nicer plots, includes ggplot2

require(MBA)
require(colorRamps)
require(lubridate)
require(reshape2)

require(oce, warn.conflicts = FALSE)

# library(lubridate)
# library(reshape2)
# library(scales)
# library(chron)
# library(ncdf4)

# Here I want to quickly test the different types of interpolation
## and compare calculated means with the published data in Pinckney et al. 2015

In [13]:
midpoint <- function(f, a, b, n){
    h = (b-a)/n
    
    result = 0
    
    for (i = 0:(n-1)){
        result = result + f((a + h/2) + i*h)}
    
    result_integration = h*result
    return(result_integration)
}


ERROR: Error in parse(text = x, srcfile = src): <text>:6:12: unexpected '='
5:     
6:     for (i =
              ^


### what do they actually say in the pub:

they integrated by the trapezoidal rule using the midpoint between sample depths and and the measured values for each sample depth

In [14]:
TotChlA <- read.csv("CARIACO Data Chl a Contour Plot.csv")
TotChlA$Date.1 <- as.POSIXct(as.character(TotChlA$Time.1), format="%m/%d/%y")
TotChlA$Date.2 <- as.POSIXct(as.character(TotChlA$Time.2), format="%m/%d/%y")

In [15]:
chla1 <- data.frame("date" = TotChlA$Date.1, "press" = TotChlA$Depth.1, "chla" = TotChlA$Chla.1)[complete.cases(data.frame("date" = TotChlA$Date.1, "press" = TotChlA$Depth.1, "chla" = TotChlA$Chla.1)),] 

In [16]:
chla2 <- data.frame("date" = TotChlA$Date.2, "press" = TotChlA$Depth.2, "chla" = TotChlA$Chla.2)[complete.cases(data.frame("date" = TotChlA$Date.2, "press" = TotChlA$Depth.2, "chla" = TotChlA$Chla.2)),] 

In [17]:
chla1_trunc <- head(chla1, n=8)
chla1_trunc

date,press,chla
1995-12-14,1,107
1995-12-14,7,139
1995-12-14,15,163
1995-12-14,25,408
1995-12-14,35,241
1995-12-14,55,88
1995-12-14,75,51
1995-12-14,100,14


In [18]:
middepth <- chla1_trunc$press[-length(chla1_trunc$press)] + diff(chla1_trunc$press)/2
midchla <- chla1_trunc$chla[-length(chla1_trunc$chla)] + diff(chla1_trunc$chla)/2

midchla1_trunc <- data.frame(middepth,midchla)

In [19]:
chla1_trunc$chla
as.numeric(stats::filter(chla1_trunc$chla, c(0.5,0.5)))

[1] 107 139 163 408 241  88  51  14

[1] 123.0 151.0 285.5 324.5 164.5  69.5  32.5    NA

In [25]:
x <- seq(0,50,10)
DFtest <- chla1_df %>%
    group_by(date) %>%
    filter(sum(!is.na(value))>2) %>%
    do(data.frame(middepth = as.numeric(stats::filter(.$depth, c(0.5,0.5))), midval = as.numeric(stats::filter(.$value, c(0.5,0.5)))))
head(DFtest, n=9)

date,middepth,midval
1995-12-14,4.0,123.0
1995-12-14,11.0,151.0
1995-12-14,20.0,285.5
1995-12-14,30.0,324.5
1995-12-14,45.0,164.5
1995-12-14,65.0,69.5
1995-12-14,87.5,32.5
1995-12-14,NA,NA
1996-01-13,4.0,1071.0


In [26]:
integrateTrapezoid(x=chla1_trunc$press, y=chla1_trunc$chla) / 1000
integrateTrapezoid(x=midchla1_trunc$middepth, y=midchla1_trunc$midchla) / 1000
integrateTrapezoid(x=chla1_trunc$press, y=chla1_trunc$chla, xmin=0, xmax=100) / 1000
integrateTrapezoid(x=midchla1_trunc$middepth, y=midchla1_trunc$midchla, xmin=0, xmax=100) / 1000

[1] 13.5385

[1] 13.12825

[1] 13.6455

[1] 14.0265

In [65]:
integrate <- function(DF, spec="trap"){
    if(spec=="trap"){
        DF_1 <- DF %>%
            group_by(date, depth) %>%
            summarise(value = mean(value))
        
        DF_int <- DF_1 %>%
            group_by(date) %>%
            do(data.frame(value_int = with(.,integrateTrapezoid(x=depth, y=value))))
        print(tail(DF_1,n=11))
        }else{
            DF_1 <- DF %>%
            group_by(date,depth) %>%
            summarise(value = mean(value))
        
        DFtest <- DF_1 %>%
            group_by(date) %>%
            filter(sum(!is.na(value))>2) %>%
            do(data.frame(middepth = as.numeric(stats::filter(.$depth, c(0.5,0.5))), midval = as.numeric(stats::filter(.$value, c(0.5,0.5)))))
        DFtest <- na.omit(DFtest)
        print(tail(DF_1,n=10))
        DF_int <- DFtest %>%
        group_by(date) %>%
        do(data.frame(value_int = with(.,integrateTrapezoid(x=middepth, y=midval))))
        print(head(DF_int))
        
    }
    return(DF_int)
}

In [66]:
prep_df_ntrplt <- function(dataframe){
    colnames(dataframe) <- c("date", "depth","value")
    return(dataframe)
}

In [67]:
chla1_df <- prep_df_ntrplt(chla1)
chla1_int_trap <- integrate(chla1_df)
chla1_int_mid <- integrate(chla1_df, spec="mid")
chla2_df <- prep_df_ntrplt(chla2)
chla2_int_trap <- integrate(chla2_df)
chla2_int_mid <- integrate(chla2_df, spec="mid")

ERROR: Error in integrateTrapezoid(x = depth, y = value): object 'depth' not found


In [57]:
mean(chla1_int_trap$value_int)/1000
sd(chla1_int_trap$value_int)/1000
mean(chla2_int_trap$value_int)/1000
sd(chla2_int_trap$value_int)/1000


(mean(chla2_int_trap$value_int) / mean(chla1_int_trap$value_int))

[1] 29.99528

[1] 26.08476

[1] 34.70573

[1] 16.91967

[1] 1.15704

In [72]:
mean(chla1_int_mid$value_int)/1000
sd(chla1_int_mid$value_int)/1000
mean(chla2_int_mid$value_int)/1000
sd(chla2_int_mid$value_int)/1000

(mean(chla2_int_mid$value_int) / mean(chla1_int_mid$value_int))

33.43/27.38

[1] 28.76678

[1] 24.69648

[1] 32.43169

[1] 15.30096

[1] 1.127401

[1] 1.220964

In [59]:
FuncTypeChlA <- read.csv("CARIACO Data for Algal Group Plots.csv")

In [60]:
FuncTypeChlA$Date <- as.POSIXct(as.character(FuncTypeChlA$Date), format="%m/%d/%y")
FuncTypeChlA$X2Date <- as.POSIXct(as.character(FuncTypeChlA$X2Date), format="%m/%d/%y")

In [61]:
diatom_chla1 <- data.frame("date" = FuncTypeChlA$Date, "press" = FuncTypeChlA$depth, 
                    "chla" = FuncTypeChlA$Diatoms_Hapto1)[complete.cases(data.frame(
                    "date" = FuncTypeChlA$Date, "press" = FuncTypeChlA$depth, "chla" = FuncTypeChlA$Diatoms_Hapto1)),] 
diatom_chla2 <- data.frame("date" = FuncTypeChlA$X2Date, "press" = FuncTypeChlA$X2depth, 
                    "chla" = FuncTypeChlA$X2Diatoms_Hapto1)[complete.cases(data.frame(
                    "date" = FuncTypeChlA$X2Date, "press" = FuncTypeChlA$X2depth, "chla" = FuncTypeChlA$X2Diatoms_Hapto1)),] 

In [62]:
diatom_chla1_df <- prep_df_ntrplt(diatom_chla1)
diatom_chla1_int_trap <- integrate(diatom_chla1_df)
diatom_chla1_int_mid <- integrate(diatom_chla1_df, spec="mid")
diatom_chla2_df <- prep_df_ntrplt(diatom_chla2)
diatom_chla2_int_trap <- integrate(diatom_chla2_df)
diatom_chla2_int_mid <- integrate(diatom_chla2_df, spec="mid")

# A tibble: 11 x 3
# Groups:   date [2]
   date                depth  value
   <dttm>              <dbl>  <dbl>
 1 2000-09-12 00:00:00    55 417.  
 2 2000-09-12 00:00:00    75 180.  
 3 2000-09-12 00:00:00   100  10.8 
 4 2000-10-10 00:00:00     1   0   
 5 2000-10-10 00:00:00     7   0   
 6 2000-10-10 00:00:00    15   0   
 7 2000-10-10 00:00:00    25   0   
 8 2000-10-10 00:00:00    35  96.6 
 9 2000-10-10 00:00:00    55  75.5 
10 2000-10-10 00:00:00    75   7.36
11 2000-10-10 00:00:00   100   0   
# A tibble: 10 x 3
# Groups:   date [2]
   date                depth  value
   <dttm>              <dbl>  <dbl>
 1 2000-09-12 00:00:00    75 180.  
 2 2000-09-12 00:00:00   100  10.8 
 3 2000-10-10 00:00:00     1   0   
 4 2000-10-10 00:00:00     7   0   
 5 2000-10-10 00:00:00    15   0   
 6 2000-10-10 00:00:00    25   0   
 7 2000-10-10 00:00:00    35  96.6 
 8 2000-10-10 00:00:00    55  75.5 
 9 2000-10-10 00:00:00    75   7.36
10 2000-10-10 00:00:00   100   0   
# A tibble: 6 x 2
# 

In [63]:
mean(diatom_chla1_int_trap$value_int)/1000
sd(diatom_chla1_int_trap$value_int)/1000
mean(diatom_chla2_int_trap$value_int)/1000
sd(diatom_chla2_int_trap$value_int)/1000


#(mean(diatom_chla1_int_trap$value_int) / mean(diatom_chla2_int_trap$value_int))

[1] 16.52969

[1] 19.65192

[1] 12.77051

[1] 15.45555

In [64]:
mean(diatom_chla1_int_mid$value_int)/1000
sd(diatom_chla1_int_mid$value_int)/1000
mean(diatom_chla2_int_mid$value_int)/1000
sd(diatom_chla2_int_mid$value_int)/1000

#(mean(diatom_chla2_int_mid$value_int) / mean(diatom_chla1_int_mid$value_int))

[1] 15.47198

[1] 18.37191

[1] 11.68914

[1] 13.85454

In [70]:
diatom_chla1_df %>%
            group_by(date) %>%
            summarise(value = mean(value))%>%
            arrange(desc(value))

date,value
1997-01-08,1524.736250
1996-04-17,1406.825714
1998-03-12,1302.983750
1997-04-16,810.326250
1997-03-13,796.015000
1997-02-23,625.522857
1996-05-10,610.081250
1999-01-12,530.821667
1996-01-13,440.416250
1997-05-10,385.951250
